In [1]:
import os
import random
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

from datetime import timedelta
from multiprocessing import cpu_count
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid

############################## Write your code ##############################
##### Optional
# Import more if you may
# Note: If there is an import error when grading, you will get F
# Hint: Stick to the often used packages
#############################################################################

In [2]:
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.benchmark = True

GPU = 0

device = torch.device(f'cuda:{GPU}' if torch.cuda.is_available() else 'cpu')

def fake_z(batch_size, noise, device):
    return torch.randn(batch_size, noise, 1, 1, device=device)

In [4]:
transform = transforms.Compose([
############################## Write your code ##############################
# Transform your data
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5) # -1 ~ 1
#############################################################################
])

train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
############################## Write your code ##############################
##### Optional
# Change BATCH_SIZE if you may
BATCH_SIZE = 2 ** 8

# Change NUM_WORKERS if you may
NUM_WORKERS = 0
#############################################################################
    
trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [ ]:
############################## Write your code ##############################
# Build Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            # 1x1 -> 4x4
            nn.ConvTranspose2d(100, 128, kernel_size=4, bias=False), 
            nn.BatchNorm2d(128),  #batch normalization
            nn.ReLU(inplace=False),
            # 4x4 -> 6x6
            nn.ConvTranspose2d(128, 64, kernel_size=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False),
            # 6x6 -> 14x14
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=False),
            # 14x14 -> 28x28 (don't use fully connected layer)
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh() # output value 범위지정
        )

    def forward(self, x):
        output = x
        return self.net(output)
#############################################################################

In [ ]:
############################## Write your code ##############################
# Build Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # 28x28 -> 14x14
            nn.Conv2d(1, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout2d(0.3), #set dropout rate
            # 14x14 -> 6x6
            nn.Conv2d(32, 64, 4, 2, 0, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Dropout2d(0.3),
            # 6x6 -> 3x3
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=False),
            # 3x3 -> 1x1 (not use fc)
            nn.Conv2d(128, 1, 4, 2, 1, bias=False),
            nn.Sigmoid(), # classification
        )

    def forward(self, x):
        output = x.view(-1, 1, 28, 28)
        output = self.net(output)
        return output.squeeze()
#############################################################################

In [ ]:
############################## Write your code ##############################
# Set your model(s), optimizer(s), and loss function(s)
criterion = nn.BCELoss()
noise = 100

discriminator = Discriminator().to(device)
generator = Generator().to(device)

gen_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
dis_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)

#############################################################################

In [ ]:
############################## Write your code ##############################
##### Optional
# Change the training process below so it would fit your model
# Note: You must print 16 generated images every 20 epochs to show progress
EPOCHS = 100

start = time.time()

for ep in range(EPOCHS):

    generator.train()
    discriminator.train()

    for image, _ in trainloader:
        batch_size = image.size(0)

        image = image.to(device)
 
        real_label = torch.full((batch_size,), 1, device=device)
        fake_label = torch.full((batch_size,), 0, device=device)

        dis_optimizer.zero_grad()

        real_output = discriminator(image)
        fake_output = discriminator(generator(fake_z(batch_size, noise, device)))

        real_loss = criterion(real_output, real_label)
        fake_loss = criterion(fake_output, fake_label)
        dis_loss = real_loss + fake_loss
  
        dis_loss.backward()
        dis_optimizer.step()

        gen_optimizer.zero_grad()
  
        fake_output = discriminator(generator(fake_z(batch_size, noise, device)))
  
        gen_loss = criterion(fake_output, real_label)
  
        gen_loss.backward()
        gen_optimizer.step()

    # Print time spent every 5 epochs
    if (ep + 1) % 5 == 0:
        print('Epoch [{}/{}] Elapsed time: {}'.format(ep + 1, EPOCHS, timedelta(seconds=time.time() - start)))

    # Generate 16 images every 20 epochs
    if (ep + 1) % 20 == 0:
        discriminator.eval()
        generator.eval()

        with torch.no_grad():
            image = make_grid(generator(fake_z(16, noise, device)).view(-1, 1, 28, 28).detach().cpu())
            image = (image + 1) / 2
            image = image.numpy()
            image = np.transpose(image, (1, 2, 0))
            plt.imshow(image)
            plt.show()
#############################################################################